In [1]:
import pandas as pd

In [2]:
import numpy as np

In [51]:
dx=pd.read_csv('train.csv')
dx

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram
...,...,...,...,...
263854,https://m.media-amazon.com/images/I/612J1R1xHl...,558806,height,5.0 centimetre
263855,https://m.media-amazon.com/images/I/61Blzh2+28...,470067,height,8.5 inch
263856,https://m.media-amazon.com/images/I/51MsegDL9V...,204245,height,43.2 centimetre
263857,https://m.media-amazon.com/images/I/510KhVw4VS...,752266,height,9.1 centimetre


In [3]:
df =pd.read_csv('20_k.csv')
df.head()

,image_link,extracted_text
0,https://m.media-amazon.com/images/I/41-OB+nXKo...,056*H10cm
1,https://m.media-amazon.com/images/I/41-OKArYAW...,THEXCEPTIONAL TOOLTOTER EX620 120LBCAPACITY 46...
2,https://m.media-amazon.com/images/I/41-ONtqnvc...,SPICE TRAiN USDA TIRE ORGANC MADEIN CLOVE POWD...
3,https://m.media-amazon.com/images/I/41-Ogd9yal...,37cm/14.56in
4,https://m.media-amazon.com/images/I/41-Osym0XH...,NaN


In [4]:
test=pd.read_csv('test.csv')
test.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [9]:
dr = df.merge(test, on='image_link', how='left')
dr.head()

,image_link,extracted_text,index,group_id,entity_name
0,https://m.media-amazon.com/images/I/41-OB+nXKo...,056*H10cm,1790,752266,depth
1,https://m.media-amazon.com/images/I/41-OB+nXKo...,056*H10cm,1791,752266,height
2,https://m.media-amazon.com/images/I/41-OKArYAW...,THEXCEPTIONAL TOOLTOTER EX620 120LBCAPACITY 46...,1792,469317,maximum_weight_recommendation
3,https://m.media-amazon.com/images/I/41-ONtqnvc...,SPICE TRAiN USDA TIRE ORGANC MADEIN CLOVE POWD...,1793,993359,item_weight
4,https://m.media-amazon.com/images/I/41-Ogd9yal...,37cm/14.56in,1794,949192,depth


In [10]:
dr.shape

(129400, 5)

In [11]:
dr.isnull().sum()

image_link             0
extracted_text    101996
index                  0
group_id               0
entity_name            0
dtype: int64

In [15]:
data=dr.drop(columns=['index','group_id'],index=1)

In [27]:
data=dr

In [28]:
data=data.drop(columns=['index','group_id'],index=1)

In [29]:
data.head()

,image_link,extracted_text,entity_name
0,https://m.media-amazon.com/images/I/41-OB+nXKo...,056*H10cm,depth
2,https://m.media-amazon.com/images/I/41-OKArYAW...,THEXCEPTIONAL TOOLTOTER EX620 120LBCAPACITY 46...,maximum_weight_recommendation
3,https://m.media-amazon.com/images/I/41-ONtqnvc...,SPICE TRAiN USDA TIRE ORGANC MADEIN CLOVE POWD...,item_weight
4,https://m.media-amazon.com/images/I/41-Ogd9yal...,37cm/14.56in,depth
5,https://m.media-amazon.com/images/I/41-Ogd9yal...,37cm/14.56in,height


In [31]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split


# Preprocess the text: Convert text to lowercase and normalize units (e.g., "1400mg" -> "1400 mg")
def preprocess_text(text):
    if isinstance(text, str):  # Check if the input is a string
        text = text.lower()
        # Add space between numbers and units (e.g., "1400mg" -> "1400 mg")
        text = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', text)
        return text
    return text  # Return the original value if it's not a string (e.g., NaN)

# Apply preprocessing to the 'extracted_text' column
data['extracted_text'] = data['extracted_text'].apply(preprocess_text)

# Define regex patterns for different entity types
patterns = {
    'item_weight': r'(\d+\.?\d*\s?(kg|g|mg|lbs|pounds|oz))',  # Matches weight patterns like "1400 mg", "2.5 kg", "5 lbs"
    'item_volume': r'(\d+\.?\d*\s?(ml|l|gallon|oz))',          # Matches volume patterns like "500 ml", "1 l"
    'wattage': r'(\d+\.?\d*\s?w)',                             # Matches wattage like "60w", "100 w"
    'voltage': r'(\d+\.?\d*\s?v)',                             # Matches voltage like "220v", "110 v"
    'height': r'(\d+\.?\d*\s?(cm|mm|m|in|inch|ft|feet))',      # Matches height like "20 cm", "5 inches"
    'width': r'(\d+\.?\d*\s?(cm|mm|m|in|inch|ft|feet))',       # Matches width like "15 cm", "6 inches"
    'depth': r'(\d+\.?\d*\s?(cm|mm|m|in|inch|ft|feet))'        # Matches depth like "10 cm", "4 inches"
}

# Function to extract the entity value based on entity name using regex
def extract_entity(entity_name, text):
    if isinstance(text, str):  # Ensure text is a string
        pattern = patterns.get(entity_name)
        if pattern:
            match = re.search(pattern, text)
            if match:
                return match.group(0)
    return None

# Apply the regex matching to the dataset
data['predicted_entity_value'] = data.apply(lambda row: extract_entity(row['entity_name'], row['extracted_text']), axis=1)

# Split data into training (80%) and testing (20%) sets
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# # Evaluate the regex-based model
# def evaluate_model(test_data):
#     correct = 0
#     total = 0
#     for _, row in test_data.iterrows():
#         predicted_value = row['predicted_entity_value']
#         true_value = row['entity_value']

#         if predicted_value and predicted_value == true_value:
#             correct += 1
#         total += 1

#     accuracy = correct / total if total > 0 else 0
#     print(f"Model Accuracy: {accuracy:.2f}")

# Evaluate the model on test data
# evaluate_model(test_data)

# Show test data with predictions
print(test_data[['extracted_text', 'entity_name','predicted_entity_value']].head())


                                           extracted_text  entity_name  \
125481                                                NaN  item_weight   
37754                                                 NaN        depth   
21739                                       18.5 cm 60 cm        depth   
12075                                                 NaN        width   
16923   specification: product name diamond core drill...      voltage   

       predicted_entity_value  
125481                   None  
37754                    None  
21739                 18.5 cm  
12075                    None  
16923                    None  


In [32]:
import re

# Expanded unit map with possible abbreviations
entity_unit_map = {
    'width': {
        'centimetre': ['cm', 'centimeter', 'centimetre'],
        'foot': ['ft', 'foot', 'feet'],
        'inch': ['in', 'inch'],
        'metre': ['m', 'meter', 'metre'],
        'millimetre': ['mm', 'millimeter', 'millimetre'],
        'yard': ['yd', 'yard']
    },
    'depth': {
        'centimetre': ['cm', 'centimeter', 'centimetre'],
        'foot': ['ft', 'foot', 'feet'],
        'inch': ['in', 'inch'],
        'metre': ['m', 'meter', 'metre'],
        'millimetre': ['mm', 'millimeter', 'millimetre'],
        'yard': ['yd', 'yard']
    },
    'height': {
        'centimetre': ['cm', 'centimeter', 'centimetre'],
        'foot': ['ft', 'foot', 'feet'],
        'inch': ['in', 'inch'],
        'metre': ['m', 'meter', 'metre'],
        'millimetre': ['mm', 'millimeter', 'millimetre'],
        'yard': ['yd', 'yard']
    },
    'item_weight': {
        'gram': ['g', r'\bgm\b', 'gram'],  # \b ensures gm is matched as a full word
        'kilogram': ['kg', 'kilogram', 'kgs'],
        'microgram': ['µg', r'\bmcg\b', 'microgram'],
        'milligram': ['mg', 'milligram'],
        'ounce': ['oz', 'ounce'],
        'pound': ['lb', 'lbs', 'pound'],
        'ton': ['t', 'ton', 'tons']
    },
    'maximum_weight_recommendation': {
        'gram': ['g', r'\bgm\b', 'gram'],
        'kilogram': ['kg', 'kilogram', 'kgs'],
        'microgram': ['µg', r'\bmcg\b', 'microgram'],
        'milligram': ['mg', 'milligram'],
        'ounce': ['oz', 'ounce'],
        'pound': ['lb', 'lbs', 'pound'],
        'ton': ['t', 'ton', 'tons']
    },
    'voltage': {
        'volt': ['v', 'volt'],
        'kilovolt': ['kv', 'kilovolt'],
        'millivolt': ['mv', 'millivolt']
    },
    'wattage': {
        'watt': ['w', 'watt'],
        'kilowatt': ['kw', 'kilowatt']
    },
    'item_volume': {
        'litre': ['l', 'litre', 'liter'],
        'millilitre': ['ml', 'millilitre', 'milliliter'],
        'cubic inch': ['cu in', 'cubic inch'],
        'cubic foot': ['cu ft', 'cubic foot'],
        'gallon': ['gal', 'gallon'],
        'pint': ['pt', 'pint'],
        'quart': ['qt', 'quart'],
        'cup': ['cup'],
        'fluid ounce': ['fl oz', 'fluid ounce'],
        'centilitre': ['cl', 'centilitre'],
        'decilitre': ['dl', 'decilitre']
    }
}

# Function to replace units in entity_value based on entity_name and unit variations
def replace_units_with_dict(row):
    """Replaces units in entity_value with values from entity_unit_map based on entity_name."""
    entity_name = row['entity_name']
    entity_value = row['predicted_entity_value']

    if entity_value is None:
        return None

    if entity_name in entity_unit_map:
        unit_map = entity_unit_map[entity_name]
        for full_unit, variations in unit_map.items():
            for variation in variations:
                # Use regex to match full unit abbreviations or words only
                pattern = rf'\b{variation}\b'
                if re.search(pattern, entity_value):
                    return re.sub(pattern, full_unit, entity_value)

    return entity_value


# Apply the function
data['entity_value_cleaned'] = data.apply(replace_units_with_dict, axis=1)

# Output the cleaned DataFrame
data

,image_link,extracted_text,entity_name,predicted_entity_value,entity_value_cleaned
0,https://m.media-amazon.com/images/I/41-OB+nXKo...,056*h10 cm,depth,10 cm,10 centimetre
2,https://m.media-amazon.com/images/I/41-OKArYAW...,thexceptional tooltoter ex620 120 lbcapacity 4...,maximum_weight_recommendation,None,None
3,https://m.media-amazon.com/images/I/41-ONtqnvc...,spice train usda tire organc madein clove powd...,item_weight,None,None
4,https://m.media-amazon.com/images/I/41-Ogd9yal...,37 cm/14.56 in,depth,37 cm,37 centimetre
5,https://m.media-amazon.com/images/I/41-Ogd9yal...,37 cm/14.56 in,height,37 cm,37 centimetre
...,...,...,...,...,...
129395,https://m.media-amazon.com/images/I/A1rVsIzEtk...,NaN,maximum_weight_recommendation,None,None
129396,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,NaN,item_weight,None,None
129397,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,NaN,maximum_weight_recommendation,None,None
129398,https://m.media-amazon.com/images/I/A1tnTUPyr7...,NaN,item_weight,None,None


In [33]:
data.isnull().sum()

image_link                     0
extracted_text            101996
entity_name                    0
predicted_entity_value    109461
entity_value_cleaned      109461
dtype: int64

In [34]:
final_data=data

In [36]:
final_data.head()

,image_link,extracted_text,entity_name,predicted_entity_value,entity_value_cleaned
0,https://m.media-amazon.com/images/I/41-OB+nXKo...,056*h10 cm,depth,10 cm,10 centimetre
2,https://m.media-amazon.com/images/I/41-OKArYAW...,thexceptional tooltoter ex620 120 lbcapacity 4...,maximum_weight_recommendation,None,None
3,https://m.media-amazon.com/images/I/41-ONtqnvc...,spice train usda tire organc madein clove powd...,item_weight,None,None
4,https://m.media-amazon.com/images/I/41-Ogd9yal...,37 cm/14.56 in,depth,37 cm,37 centimetre
5,https://m.media-amazon.com/images/I/41-Ogd9yal...,37 cm/14.56 in,height,37 cm,37 centimetre


In [37]:
final_data = final_data.drop(columns=['image_link', 'extracted_text', 'entity_name', 'predicted_entity_value'])

In [38]:
final_data.head()

,entity_value_cleaned
0,10 centimetre
2,None
3,None
4,37 centimetre
5,37 centimetre


In [47]:
# Reset the index to start from 1 and rename the index column to 'index'
final_data.index = final_data.index + 1
final_data.to_csv('output_new.csv', index=True, index_label='index')

In [48]:
df=pd.read_csv('output_new.csv')
df.head()

,index,entity_value_cleaned
0,2,10 centimetre
1,4,NaN
2,5,NaN
3,6,37 centimetre
4,7,37 centimetre


In [49]:
import pandas as pd
import numpy as np

# Assuming 'final_data' is your existing DataFrame

# Create the full range of indices you want to have in the DataFrame
full_index = range(1, final_data.index.max() + 1)

# Reindex the DataFrame to include all indices in the range
# Missing indices will be filled with NaN by default
final_data_full = final_data.reindex(full_index)

# Optional: If you want to fill missing values with something other than NaN, like an empty string or zero:
# final_data_full.fillna(value='default_value', inplace=True)

# Save to CSV, ensuring the index starts from 1
final_data_full.to_csv('output_res_new.csv', index=True, index_label='index')

print("Data with all indices included:")
print(final_data_full)


Data with all indices included:
       entity_value_cleaned
1                       NaN
2             10 centimetre
3                       NaN
4                      None
5                      None
...                     ...
129397                 None
129398                 None
129399                 None
129400                 None
129401                 None

[129401 rows x 1 columns]


In [50]:
df=pd.read_csv('output_res_new.csv')
df.head()

,index,entity_value_cleaned
0,1,NaN
1,2,10 centimetre
2,3,NaN
3,4,NaN
4,5,NaN


In [52]:
df.rename(columns={'entity_value_cleaned': 'prediction'}, inplace=True)
df.head()

,index,prediction
0,1,NaN
1,2,10 centimetre
2,3,NaN
3,4,NaN
4,5,NaN
